In [ ]:
pip install nfl_data_py

In [ ]:
pip install duckdb

In [66]:
import nfl_data_py as nfl  # NFL data retrieval and analysis (via https://pypi.org/project/nfl-data-py/)
import pandas as pd  # Data manipulation and analysis
import duckdb # Used to write SQL inside Python script

In [68]:
# Make dataframe output easier to read
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Allow horizontal scrolling
pd.set_option('display.max_rows', None)  # Show all rows

years = [2020, 2021, 2022, 2023, 2024]

# Create an empty list to store DataFrames for each year
df_game_log_list = []

# Loop through each year, read the data, and append it to the list
for year in years:
    url = f'https://github.com/nflverse/nflverse-data/releases/download/pbp/play_by_play_{year}.csv.gz'
    
    # Read the data for the given year
    df_game_log_year = pd.read_csv(url, compression='gzip', low_memory=False)

    # Filter for only 'kickoff' play types
    df_game_log_year = df_game_log_year[df_game_log_year['play_type'] == 'kickoff']

    # Retain columns that have relevance to kickoffs
    df_game_log_year = df_game_log_year[['season','game_id','drive','series','series_result','fixed_drive_result','desc','weather','roof','surface','temp','wind',
                                         'kicker_player_id','kickoff_returner_player_id','penalty','return_team','return_yards','penalty_player_id','penalty_type',
                                         'penalty_yards','end_yard_line','kickoff_inside_twenty','kickoff_in_endzone','kickoff_out_of_bounds','kickoff_downed',
                                         'kickoff_fair_catch','kick_distance','fumble_lost','drive_start_yard_line','touchdown','defteam','play_type','play_deleted',
                                         'solo_tackle_1_player_id','posteam_type','penalty_team','game_half','own_kickoff_recovery','game_seconds_remaining',
                                         'half_seconds_remaining']]
    
    # Append the filtered DataFrame to the list
    df_game_log_list.append(df_game_log_year)

# Concatenate all the DataFrames for each year into a single DataFrame
df_game_log = pd.concat(df_game_log_list)

# Save NFL players and teams to dataframe
df_players = nfl.import_seasonal_rosters(years) # Import NFL rosters for each year requested
df_teams = nfl.import_team_desc() # Import NFL team information

df_teams

2021 done.
2022 done.
2023 done.


NameError: name 'Error' is not defined

In [64]:
# Use SQL to create main kickoff data set
kickoffs = duckdb.sql("""
                         select a.season,
                                a.game_id,
                                a.drive,
                                a.series,
                                a.series_result,
                                a.fixed_drive_result,
                                a.desc,
                                a.weather,
                                a.roof,
                                a.surface,
                                a.temp,
                                a.wind,
                                a.kicker_player_id,
                                a.kickoff_returner_player_id,
                                a.defteam as kicking_team,
                                c.team_name as kicking_team_name,
                                a.return_team,
                                b.team_name as return_team_name,
                                case
                                  when a.posteam_type='home' then 'Home'
                                  when a.posteam_type='away' then 'Away'
                                end as return_team_location,
                                'https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/' || a.return_team || '.png&h=200&w=200' as team_logo_espn,
                                a.end_yard_line,
                                a.kickoff_inside_twenty,
                                a.kickoff_in_endzone,
                                a.kickoff_out_of_bounds,
                                a.kickoff_downed,
                                a.kickoff_fair_catch,
                                a.kick_distance,
                                case
                                  when lower(a.desc) like '%injur%' then 1 else 0
                                end as injury,
                                a.fumble_lost,
                                a.drive_start_yard_line,
                                a.touchdown,
                                case
                                  when touchdown=1 then 100
                            	  when a.drive_start_yard_line not like '%' || return_team || '%' then 50 + (50 - cast(replace(a.drive_start_yard_line,defteam||' ','') as float))
                            	  else cast(replace(a.drive_start_yard_line,return_team||' ','') as float)
                                end as yardline_100,
                                case
                                  when d.position='K' then 1 else 0 
                                end as solo_tackle_by_kicker,
                                a.penalty_team,
                                case
                                  when a.return_team=a.penalty_team and a.penalty=1 then 'Receiving Team'
                                  when a.return_team<>a.penalty_team and a.penalty=1 then 'Kicking Team'
                                end as penalty_by_team,
                                a.penalty,
                                a.penalty_player_id,
                                a.penalty_type,
                                coalesce(a.penalty_yards,0) as penalty_yards,
                                case
                                  when a.drive=min(a.drive) over(partition by a.season,a.game_id,a.game_half) then 1
                                  else 0
                                end as first_drive_flag,
                                case
                                  when a.desc like '%onside%' then 1 else 0 
                                end as onside_kick,
                                case
                                  when a.desc like '%onside%' and own_kickoff_recovery=1 then 1
                                  else 0
                                end as onside_kick_successful,
                                game_seconds_remaining,
                                half_seconds_remaining
                                
                         from df_game_log a inner join
                              df_teams b on a.return_team = b.team_abbr inner join
                              df_teams c on a.defteam = c.team_abbr left join
                              df_players d on a.solo_tackle_1_player_id=d.player_id and
                                              a.season = d.season
                            
                         where a.play_type='kickoff' and
                               a.play_deleted=0 and
                               a.return_team is not null
                         """).df()

# Use SQL to summarize/aggregate main kickoff data set, kickoffs, at the season level
kickoffs_agg = duckdb.sql("""select 
                                season,
                                count(*) as number_kickoffs,
                                avg(yardline_100) as avg_starting_position,
                                avg(case when kickoff_returner_player_id is not null then yardline_100 end) as avg_starting_position_returns,
                                avg(case when kickoff_returner_player_id is not null and penalty=0 then yardline_100 end) as avg_starting_position_touchbacks,
                                sum(case when touchdown=1 then 1 else 0 end)/(count(*)*1.0) as touchdown_return_rate,
                                sum(case when fixed_drive_result in ('Field goal','Touchdown') then 1 else 0 end)/(count(*)*1.0) as touchdown_rate_on_drives_following_kickoffs,
                                sum(case when fixed_drive_result in ('Field goal','Touchdown') and first_drive_flag = 1 then 1 else 0 end)/(sum(first_drive_flag)*1.0) as touchdown_rate_on_first_drives_of_half,
                                sum(injury)/(count(*)*1.0) as injury_rate,
                                sum(injury) as injuries,
                                sum(case when kickoff_returner_player_id is not null then 1 else 0 end)/(count(*)*1.0) as return_rate,
                                sum(penalty) as penalties,
                                sum(penalty)/(count(*)*1.0) as penalty_rate
                                
                                from
                                kickoffs
                                
                                group by
                                season""").df()

# Use SQL to summarize/aggregate main kickoff data set, kickoffs, at the team level
kickoffs_team_agg = duckdb.sql("""select 
                                return_team_name,
                                team_logo_espn as url,
                                count(*) as number_kickoffs,
                                avg(yardline_100) as avg_starting_position,
                                avg(case when kickoff_returner_player_id is not null then yardline_100 end) as avg_starting_position_returns,
                                avg(case when kickoff_returner_player_id is not null and penalty=0 then yardline_100 end) as avg_starting_position_touchbacks,
                                sum(case when touchdown=1 then 1 else 0 end)/(count(*)*1.0) as touchdown_return_rate,
                                sum(case when fixed_drive_result in ('Field goal','Touchdown') then 1 else 0 end)/(count(*)*1.0) as touchdown_rate_on_drives_following_kickoffs,
                                sum(case when fixed_drive_result in ('Field goal','Touchdown') and first_drive_flag = 1 then 1 else 0 end)/(sum(first_drive_flag)*1.0) as touchdown_rate_on_first_drives_of_half,
                                sum(injury)/(count(*)*1.0) as injury_rate,
                                sum(injury) as injuries,
                                sum(case when kickoff_returner_player_id is not null then 1 else 0 end)/(count(*)*1.0) as return_rate,
                                sum(penalty) as penalties,
                                sum(penalty)/(count(*)*1.0) as penalty_rate
                                
                                from
                                kickoffs a inner join
                                (select max(season) as max_season
                                   from kickoffs) b on a.season = b.max_season
                                
                                group by
                                return_team_name,
                                team_logo_espn""").df()

# Print team aggregate data set
kickoffs_team_agg

,return_team_name,url,number_kickoffs,avg_starting_position,avg_starting_position_returns,avg_starting_position_touchbacks,touchdown_return_rate,touchdown_rate_on_drives_following_kickoffs,touchdown_rate_on_first_drives_of_half,injury_rate,injuries,return_rate,penalties,penalty_rate
0,Seattle Seahawks,https://a.espncdn.com/combiner/i?img=/i/teamlo...,90,26.528090,30.240000,29.625000,0.000000,0.411111,0.500000,0.011111,1.0,0.288889,2.0,0.022222
1,Pittsburgh Steelers,https://a.espncdn.com/combiner/i?img=/i/teamlo...,84,26.035714,27.035714,27.666667,0.000000,0.250000,0.333333,0.011905,1.0,0.333333,1.0,0.011905
2,Washington Commanders,https://a.espncdn.com/combiner/i?img=/i/teamlo...,107,25.266667,26.272727,26.272727,0.000000,0.261682,0.222222,0.000000,0.0,0.224299,0.0,0.000000
3,Miami Dolphins,https://a.espncdn.com/combiner/i?img=/i/teamlo...,87,25.586207,27.000000,27.636364,0.000000,0.367816,0.388889,0.000000,0.0,0.264368,2.0,0.022989
4,Tampa Bay Buccaneers,https://a.espncdn.com/combiner/i?img=/i/teamlo...,88,24.579545,23.318182,25.052632,0.000000,0.386364,0.473684,0.011364,1.0,0.250000,3.0,0.034091
5,Arizona Cardinals,https://a.espncdn.com/combiner/i?img=/i/teamlo...,94,25.095745,24.526316,25.117647,0.000000,0.351064,0.470588,0.000000,0.0,0.202128,3.0,0.031915
6,Carolina Panthers,https://a.espncdn.com/combiner/i?img=/i/teamlo...,88,25.477273,26.416667,27.666667,0.000000,0.238636,0.176471,0.000000,0.0,0.272727,5.0,0.056818
7,Tennessee Titans,https://a.espncdn.com/combiner/i?img=/i/teamlo...,92,25.108696,25.312500,25.133333,0.000000,0.315217,0.368421,0.000000,0.0,0.347826,2.0,0.021739
8,Cincinnati Bengals,https://a.espncdn.com/combiner/i?img=/i/teamlo...,87,25.068966,25.352941,27.133333,0.000000,0.287356,0.500000,0.011494,1.0,0.195402,2.0,0.022989
9,Chicago Bears,https://a.espncdn.com/combiner/i?img=/i/teamlo...,81,25.259259,25.840000,27.130435,0.000000,0.234568,0.352941,0.000000,0.0,0.308642,2.0,0.024691


In [ ]:
# Use SQL to create main kickoff data set
kickoffs = duckdb.sql("""
                         select b.position, 
                                a.desc, 
                                a.return_team, 
                                c.team_name as return_team_name,
                                a.defteam as kicking_team,
                                d.team_name as kicking_team_name
                                
                         from df_game_log a inner join
                              df_players b on a.season=b.season and
                                              a.solo_tackle_1_player_id=b.player_id inner join
                              df_teams c on a.return_team=c.team_abbr inner join
                              df_teams d on a.defteam=d.team_abbr
                            
                         where a.play_type='kickoff'
                         """).df()

kickoffs